In [1]:
import folium
import json
from datascience import *
import pandas as pd

In [2]:
#Note, tables is not working for this!!
# data = Table.read_table('data/sf_oak_TimeSkims_AM.csv')
# data_1 = data.where('orig',1)
# data_1


data = pd.read_csv('data/sf_oak_TimeSkims_AM.csv')
data_1 = data[data['orig']==1]

,orig,dest,da,daToll,s2,s2Toll,s3,s3Toll,walk,bike,wTrnW,dTrnW,wTrnD
0,1,1,5.92,5.92,5.92,5.92,5.92,5.92,2.4,0.60,-999.00,-999.00,-999.00
1,1,2,6.25,6.25,6.25,6.25,6.25,6.25,4.8,1.20,7.20,5.75,3.82
2,1,3,6.70,6.70,6.70,6.70,6.70,6.70,8.8,2.20,9.70,5.75,4.42
3,1,4,6.66,6.66,6.66,6.66,6.66,6.66,8.2,2.05,6.12,4.18,3.76
4,1,5,7.34,7.34,7.34,7.34,7.34,7.34,13.6,3.40,7.46,5.25,4.87
5,1,6,7.22,7.22,7.22,7.22,7.22,7.22,19.4,4.85,13.04,10.43,6.10
6,1,7,7.40,7.40,7.40,7.40,7.40,7.40,19.6,4.90,10.78,8.42,5.77
7,1,8,7.31,7.31,7.31,7.31,7.31,7.31,23.0,5.75,10.04,7.36,7.06
8,1,9,8.67,8.67,8.67,8.67,8.67,8.67,31.2,7.80,12.98,11.28,9.54
9,1,10,8.55,8.55,8.55,8.55,8.55,8.55,31.8,7.95,13.39,9.65,9.63


In [20]:
gjd = json.load(open('data/SF_Oak_TAZs.geojson'))

gjd_str = json.dumps(gjd)

mapa = folium.Map(width=650, height=500, zoom_start=11,
                  location=[37.8062449,-122.371983])



mapa.geo_json(geo_str=gjd_str, data=data_1,
              columns=['dest', 'da'],
              threshold_scale=[5,10,15,20,30],
              key_on='feature.properties.TAZ1454',
              fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,)


gjd['features']= [gjd['features'][0]]
gjd_str = json.dumps(gjd)

mapa.geo_json(geo_str=gjd_str,fill_color='Red', 
              fill_opacity=0.7, line_opacity=0.5,)

mapa.create_map('test_map.html')
mapa

